In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import RSRAE

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
        config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
detector = RSRAE(
    encoder_dims=[500, 450, 400, 350, 300],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    rsr_dim=250,
    batch_norm=True,
)

In [4]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [5]:
# Training
epochs = 10
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-04-24 23:30:59,569 [INFO] __main__: Start training the model...
2024-04-24 23:30:59,569 [INFO] __main__: Epoch: 1
2024-04-24 23:30:59,932 [INFO] __main__: Loss: 526.431702
2024-04-24 23:31:02,076 [INFO] __main__: Loss: 145.038376
2024-04-24 23:31:03,980 [INFO] __main__: Loss: 85.928146
2024-04-24 23:31:05,925 [INFO] __main__: Loss: 64.805748
2024-04-24 23:31:07,785 [INFO] __main__: Loss: 49.371063
2024-04-24 23:31:07,944 [INFO] __main__: Epoch: 2
2024-04-24 23:31:07,967 [INFO] __main__: Loss: 47.522552
2024-04-24 23:31:09,929 [INFO] __main__: Loss: 39.259193
2024-04-24 23:31:11,920 [INFO] __main__: Loss: 32.623493
2024-04-24 23:31:13,769 [INFO] __main__: Loss: 29.601597
2024-04-24 23:31:15,742 [INFO] __main__: Loss: 26.740170
2024-04-24 23:31:15,897 [INFO] __main__: Epoch: 3
2024-04-24 23:31:15,915 [INFO] __main__: Loss: 25.979897
2024-04-24 23:31:17,761 [INFO] __main__: Loss: 24.746241
2024-04-24 23:31:19,754 [INFO] __main__: Loss: 22.738628
2024-04-24 23:31:21,775 [INFO] __main_

In [6]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-04-24 23:32:19,089 [INFO] __main__: Start evaluating the model...
2024-04-24 23:32:20,626 [INFO] __main__: AUROC Score: 0.42992413986266703
2024-04-24 23:32:20,626 [INFO] __main__: Completed the evaluation of the model!
